In [ ]:
import csv
import math
import numpy as np


# TD5 - DSIA - Yan PODOLAK et Kévin LIM - TDK

class Fleur:
    def __init__(self, sepal_length=0, sepal_width=0, petal_length=0, petal_width=0, label=''):
        self.sepal_length = float(sepal_length)
        self.sepal_width = float(sepal_width)
        self.petal_length = float(petal_length)
        self.petal_width = float(petal_width)
        if label == '':
            self.label = 'Inconnu'
        else:
            self.label = label

    def __str__(self):
        return f"({self.sepal_length},{self.sepal_width},{self.petal_length},{self.petal_width},{self.label})"


def CreationFleur(data):
    liste_fleur = []
    for i in range(len(data)):
        fl = Fleur(data[i][0], data[i][1], data[i][2], data[i][3], data[i][4])
        #print(fl.label)
        liste_fleur.append(fl)

    return liste_fleur

def Creationv2(data):
    liste_fleur = []
    for i in range(len(data)):
        fl = Fleur(data[i][0], data[i][1], data[i][2], data[i][3])
        #print(fl.label)
        liste_fleur.append(fl)
    return liste_fleur


def ImportDataset(nom):
    tab = []
    with open(nom, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in spamreader:
            row = ','.join(row)
            data = row.split(';')
            tab.append(data)
            # print(data)
            # input()
    # print(tab)
    #print(tab)
    return tab


def calc_min(liste_fleur, objet):
    calcul = 1000
    if objet == "sepal_length":
        for i in range(len(liste_fleur)):
            calcul = min(calcul, liste_fleur[i].sepal_length)

    if objet == "sepal_width":
        for i in range(len(liste_fleur)):
            calcul = min(calcul, liste_fleur[i].sepal_width)

    if objet == "petal_length":
        for i in range(len(liste_fleur)):
            calcul = min(calcul, liste_fleur[i].petal_length)

    if objet == "petal_width":
        for i in range(len(liste_fleur)):
            calcul = min(calcul, liste_fleur[i].petal_width)

    return calcul


def calc_max(liste_fleur, objet):
    calcul = -1000
    if objet == "sepal_length":
        for i in range(len(liste_fleur)):
            calcul = max(calcul, liste_fleur[i].sepal_length)

    if objet == "sepal_width":
        for i in range(len(liste_fleur)):
            calcul = max(calcul, liste_fleur[i].sepal_width)

    if objet == "petal_length":
        for i in range(len(liste_fleur)):
            calcul = max(calcul, liste_fleur[i].petal_length)

    if objet == "petal_width":
        for i in range(len(liste_fleur)):
            calcul = max(calcul, liste_fleur[i].petal_width)

    return calcul


def Norm(liste_fleur, val, arg):
    return (val - calc_min(liste_fleur, arg)) / (calc_max(liste_fleur, arg) - calc_min(liste_fleur, arg))


def Distance(liste_fleur, fl1, fl2):
    return math.sqrt(
        (Norm(liste_fleur, fl1.sepal_length, "sepal_length") - Norm(liste_fleur, fl2.sepal_length, "sepal_length")) ** 2
        + (Norm(liste_fleur, fl1.sepal_width, "sepal_width") - Norm(liste_fleur, fl2.sepal_width, "sepal_width")) ** 2
        + (Norm(liste_fleur, fl1.petal_length, "petal_length") - Norm(liste_fleur, fl2.petal_length,
                                                                      "petal_length")) ** 2
        + (Norm(liste_fleur, fl1.petal_width, "petal_width") - Norm(liste_fleur, fl2.petal_width, "petal_width")) ** 2)


def sortSecond(val):
    return val[1]

def sortFourth(val):
    return val[4]

#Permet de travailler avec des valeurs de 0 à 9 au lieu de travailler sur les labels
def replace(val):
    if val == 'A':
        return 0
    elif val == 'B':
        return 1
    elif val == 'C':
        return 2
    elif val == 'D':
        return 3
    elif val == 'E':
        return 4
    elif val == 'F':
        return 5
    elif val == 'G':
        return 6
    elif val == 'H':
        return 7
    elif val == 'I':
        return 8
    elif val == 'J':
        return 9
    else:
        return 10


def MConfusion(liste_fleur,liste_predic, k):
    
    #Création de la matrice vide de confusion
    matrice_confu = np.zeros((10, 10))
    print("Création de la matrice de confusion...")
    # Création de la nouvelle liste de fleur contenant la donnée à prédire
    liste_fleur_kNN = []

    # Parcourir la liste de fleur a prédire
    for i in range(len(liste_predic)):
        
        # Récupérer le vrai labal de la fleur a prédire
        vrai_label = liste_predic[i].label
        
        # Récupérer la liste avec les données d'entraienement
        liste2 = liste_fleur
        
        # Ajouteur la fleur à prédire dans la liste
        liste_fleur_kNN.append(liste_predic[i])
        
        # Prédire le label de cette fleur avec kNN et la liste d'entrainement
        liste_fleur_kNN[i].label = kNN(liste_fleur_kNN[i], liste2, k)
        
        # Remplissage de la matrice de confusion en fonction du label prédit
        matrice_confu[replace(liste_fleur_kNN[i].label), replace(vrai_label)] += 1;
        print("Fleur : " + str(i+1) + "/"+ str(len(liste_predic)+1))

    # Calcul 
    erreur = 0
    for i in range(len(matrice_confu)):
        for j in range(len(matrice_confu[i])):
            if i != j:
                erreur += matrice_confu[i][j]

    for i in range(len(matrice_confu)):
        for j in range(len(matrice_confu[i])):
            print(matrice_confu[i][j], end=" ")
        print()
    print("Pour k=", k, " l'erreur est: ", erreur)
    return erreur

def CalculdeK(liste_fleur, fichier):
    #Teste les k et compare les matrices de confusion pour trouver le meilleur k. On a choisi k=2 après essais.
    index = 0
    erreur = 150
    for i in range(2, 20):  # revoir le k
        liste_fleur = CreationFleur(fichier)
        e = MConfusion(liste_fleur, i)
        print("Pour k=", i, " l'erreur est: ", e)
        if e < erreur:
            erreur = e
            index = i
        if e == 0:
            index = i
            break;
    return index


def kNN(fleur, liste_fleur, k):
    #On calcule la distance entre la fleur et toutes les fleurs du fichier d'entrainement
    fleur_et_distance = []
    for i in range(1, len(liste_fleur)):
        dist = Distance(liste_fleur, fleur, liste_fleur[i])
        temp = [liste_fleur[i].label, dist]
        fleur_et_distance.append(temp)
    #On trie les fleurs par distance
    fleur_et_distance.sort(key=sortSecond)

    #Pour les k fleurs les plus près, on compte le nombre de chaque fleur : 0=>A,1=>B...
    count_label = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for j in range(k):
        count_label[replace(fleur_et_distance[j][0])] += 1

    #On trouve le type de fleur le plus présent dans les k premières
    max = count_label[0]
    label = 0
    for i in range(0, len(count_label)):
        if count_label[i] > max:
            max = count_label[i]
            label = i
    if label == 0:
        label_nouvellefleur = 'A'
    if label == 1:
        label_nouvellefleur = 'B'
    if label == 2:
        label_nouvellefleur = 'C'
    if label == 3:
        label_nouvellefleur = 'D'
    if label == 4:
        label_nouvellefleur = 'E'
    if label == 5:
        label_nouvellefleur = 'F'
    if label == 6:
        label_nouvellefleur = 'G'
    if label == 7:
        label_nouvellefleur = 'H'
    if label == 8:
        label_nouvellefleur = 'I'
    if label == 9:
        label_nouvellefleur = 'J'
    return label_nouvellefleur

def predict_data_export(liste_fleur,liste_predic, k):
    #tableau des labels prédits
    label = []
    #On prédit le label de chaque fleur de la liste à prédire
    cpt = 0
    for i in liste_predic:
        cpt+=1
        label_predict = kNN(i, liste_fleur, k)
        label.append(label_predict)
        print("Fleur : " + str(cpt) + "/"+ str(len(liste_predic)))
    #Création du fichier
    file = open("PODOLAK_LIM.txt", "w")
    #Ecriture du fichier avec les labels prédits
    for i in range(len(label)):
        saut = ""
        if i < len(label)-1:
            saut="\n"
        file.write(label[i]+saut)
    file.close()

if __name__ == '__main__':
    # Initialisation

    # Importation des données d'entrainement
    data = ImportDataset('data.csv')
    
    # Séléction de 50 fleurs de chaque catégorie dans la donnée d'entrainement
    data.sort(key=sortFourth)
    new_data = []
    letters =[0,0,0,0,0,0,0,0,0,0] 
    for i in range (len(data)):
        letter = data[i][4]
        if letters[replace(letter)]<150:
          new_data.append(data[i])
          letters[replace(letter)]+=1

    # Creation des objets avec les données obtenues précédemment
    liste_fleur = CreationFleur(new_data)
    
    # Permet de calculer le meilleur k pour ce jeu de données
    #k = CalculdeK(liste_fleur, new_data)

    # Importation des données à prédire
    data2 = ImportDataset('finalTest.csv')
   
    # Création de la liste d'objets à prédire
    liste_predic = Creationv2(data2)
    
    #Calcul de la matrice de confusion (erreurs)
    #MConfusion(liste_fleur,liste_predic,2)
    
    #Exportation des valeurs prédite dans un fichier
    predict_data_export(liste_fleur,liste_predic, 2)

